In [ ]:
#this function fills NaN values with mean value for continuous features and mode value for nominal/ordinal features

def impute_nan_values(df):  
    continuous_features = ['EINGEZOGENAM_HH_JAHR' , 'MIN_GEBAEUDEJAHR', 'KBA13_ANZAHL_PKW', 'ANZ_HAUSHALTE_AKTIV','VERDICHTUNGSRAUM', 'ANZ_STATISTISCHE_HAUSHALTE']
    
    #fill continuous features nan with mean value
    df[continuous_features] = df[continuous_features].fillna(np.mean(df[continuous_features]))

    #list of categorical/ordinal features
    all_features_except_continuous = df.columns.to_list()
    for i in continuous_features:
        all_features_except_continuous.remove(i)

    #fill categorical features nan with most frequent values
    for column in df[all_features_except_continuous].columns:
        df[column].fillna(df[column].mode()[0], inplace=True)
    
    return df

In [ ]:
def cleaning_data(train_data):
    columns_to_drop = ['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992', 'KK_KUNDENTYP']
    train_data.drop(columns = columns_to_drop, inplace=True)
    k = 0
    for i,j in var_unknown.items():
        if i in columns_to_remove:
            continue
        if i in columns_to_rename:
            train_data[columns_renamed[k]] = train_data[columns_renamed[k]].replace(j, np.nan)
            continue
        train_data[i] = train_data[i].replace(j, np.nan)
    #'RELAT_AB' need to replace 9 with nan
    train_data['RELAT_AB'] = train_data['RELAT_AB'].replace([9], np.nan)
    columns_to_drop_1 = ['AGER_TYP', 'KBA05_BAUMAX', 'TITEL_KZ']
    train_data.drop(columns = columns_to_drop_1, inplace=True)

    #dropping 'LP_STATUS_GROB', 'LP_FAMILIE_GROB' because similar to 'LP_STATUS_FEIN' and 'LP_FAMILIE_FEIN' 
    train_data.drop(columns = ['LP_STATUS_GROB', 'LP_FAMILIE_GROB'], inplace=True)
    #dropping LNR
    train_data.drop(columns = ['LNR'], inplace=True)

    #reengineering features
    #PLZ8_BAUMAX, how many family homes
    train_data['PLZ8_BAUMAX'] = train_data['PLZ8_BAUMAX'].replace([5], 0)
    #if person lives in business building or not
    train_data['PLZ8_BAUMAX_BIZ'] = train_data['PLZ8_BAUMAX'].replace([0,1,2,3,4], [1,0,0,0,0])
    #main of avant-garde
    train_data['PRAEGENDE_MAIN_OR_AVANT'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [0,1,0,1,0,1,1,0,1,0,1,0,1,0,1])
    #40ies,50ies,60ies,70ies,80ies,90ies movements
    train_data['PRAEGENDE_JUGENDJAHRE'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [1,1,2,2,3,3,3,4,4,5,5,5,5,6,6])

    #CAMEO_INTL
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].astype(float)
    train_data['CAMEO_DEUG_2015'] = train_data['CAMEO_DEUG_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_DEUG_2015']  = train_data['CAMEO_DEUG_2015'].astype(float)
    #household income
    train_data['CAMEO_HOUSEHOLD'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                  41, 42, 43, 44, 45, 51, 52, 53, 54, 55],[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5])
    #family type 
    train_data['CAMEO_FAMILY_TYPE'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                   41, 42, 43, 44, 45, 51, 52, 53, 54, 55], [1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5])

    #OST_WEST_KZ encode 'W' to 0 and '0' to 1
    train_data['OST_WEST_KZ'] = train_data['OST_WEST_KZ'].replace(['W', 'O'], [0,1]) 
    #one hot encoding of 'ANREDE_KZ'
    train_data = pd.get_dummies(train_data, columns=['ANREDE_KZ'], drop_first=True)


    train_data.drop(columns = ['CAMEO_INTL_2015'], inplace=True)
    #for WOHNLAGE, replace 0 with nan
    train_data['WOHNLAGE'] = train_data['WOHNLAGE'].replace([0], np.nan)
    #create new WOHNLAGE feature, urban or rural
    train_data['WOHNLAGE_REGION'] = train_data['WOHNLAGE'].replace([1,2,3,4,5,7,8],[0,0,0,0,0,1,1])

    #dropping CAMEO_DEU, too many categories 
    train_data.drop(columns = ['CAMEO_DEU_2015'], inplace=True)
    #dropping LP_LEBENSPHASE_FEIN, too many categories
    train_data.drop(columns = ['LP_LEBENSPHASE_FEIN'], inplace=True)
    #dropping D19_LETZTER_KAUF_BRANCHE, duplicated information
    train_data.drop(columns=['D19_LETZTER_KAUF_BRANCHE'], inplace=True)

    #converting 'EINGEFUEGT_AM' to total months
    eingefuegt = np.empty(shape=(train_data.shape[0],1))
    k = 0
    train_data['EINGEFUEGT_AM'] = train_data['EINGEFUEGT_AM'].fillna(-1)
    for i in train_data['EINGEFUEGT_AM']:
        if i != -1:

            yrmthday, zeros = i.split()
            mth_1 = (2019 - (int(yrmthday[0:4]) + 1)) * 12
            mth_2 = 12 - int(yrmthday[5:6])
            mth_3 = (30 - int(yrmthday[8:10])) / 30
            mth_total = mth_1 + mth_2 + mth_3
            eingefuegt[k] = mth_total
        else:
            eingefuegt[k] = -1
        k += 1




    train_data.drop(columns =['EINGEFUEGT_AM'] , inplace=True)
    train_data['EINGEFUEGT_MODIFIED'] = pd.DataFrame(eingefuegt)

    train_data['EINGEFUEGT_MODIFIED'] = train_data['EINGEFUEGT_MODIFIED'].replace([-1],train_data['EINGEFUEGT_MODIFIED'].mode().values)
    
    return train_data


In [ ]:
#before that PCA analysis is still useful for identifying interesting features, so we will conduct PCA here
def pca_fit(df):
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import normalize
    #normalize data
    df = normalize(df)
    #fit PCA on data
    pca = PCA().fit(df)
    
    #only select components that cumulatively exceed 0.9 explaiend variance ratio
    k = 0
    m = 1
    for i in pca.explained_variance_ratio_:
        k += i
        if k > 0.9:
            break
        m += 1

    no_comp = m
    
    #fit PCA again on data with the desired number of component
    df = PCA(n_components=m).fit_transform(df)
    return pca, df

pca, train_data_transformed = pca_fit(train_data)

#Next we will find interesting variables identified by PCA

# the set should contain the variable's string name (as found in X.colummns)
def find_interesting_variables(pca, length_pca_component, top_length, feature_names):
    interesting_variables = set()
 
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import normalize
    
    #how many PCA component to analyse
    component = pca.components_[0:length_pca_component]
    
    #how many interesting variables sorted by most interesting to least interesting to return
    for i in range(len(component)):
    
        feature_with_magnitude = dict(zip(feature_names, component))
        sorted_feature_with_magnitude = sorted(feature_with_magnitude.items(), key=lambda item: item[1])
        j = 0
        for k,l in  sorted_feature_with_magnitude.items():
            if j == top_length:
                break
            interesting_variables.add(k)
            j += 1
                    
    return interesting_variables

In [ ]:
#function to fit K-Means algortihm on dataset and then predict the clusters that dataset belong to
def kmeans_segment(df, df2, no_cluster):
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=no_cluster , n_init=no_cluster+3, verbose=3, max_iter=500)
    km.fit(df)
    predicted_cluster_azdias = km.predict(df)
    predicted_cluster_customer = km.predict(df2)
    
    return predicted_cluster_azdias, predicted_cluster_customer

#function to count how many examples are contained in each cluster
def segmentation_count(array, count):
    segment_count = {}
    for i in range(count):
        segment_count[i] = np.count_nonzero(array==i)
    return segment_count

In [ ]:
def objective(space):

    global ITERATION
    ITERATION += 1
    start = timer()
    global prediction_all

    xgbcl = xgb.XGBClassifier(colsample_bytree =  space['colsample_bytree'], gamma = space['gamma'], learning_rate = space['learning_rate'], max_depth = space['max_depth'], min_child_weight = space['min_child_weight'], n_estimators = space['n_estimators'], reg_alpha = space['reg_alpha'], scale_pos_weight = space['scale_pos_weight'], subsample = space['subsample'], tree_method = 'gpu_hist', sampling_method = 'gradient_based')

    # execute cross-validation scoring, I will used stratified cross-validation and 3 cross-validation folds
    # scoring will be roc_auc since this is an imbalanced dataset
    cv = cross_val_score(
        estimator=xgbcl,
        X=mailout_train.iloc[:,:-1],
        y=mailout_train.iloc[:,-1],
        verbose=True,
        n_jobs=-1,
        cv=StratifiedKFold(n_splits=3, shuffle=False, random_state=None),
        scoring='roc_auc',
    )
    #this fit the XGBClassifier to the MAILOUT TRAIN and fitted model is used to predict on MAILOUT TEST and predictions are concatenated to predictions_all variable
    xgbcl.fit(mailout_train.iloc[:,:-1], mailout_train.iloc[:,-1])
    predictions_xgbcl = (xgbcl.predict_proba(mailout_test))[:, 1]
    predictions_all = pd.concat([predictions_all, predictions_xgbcl], axis=1)
    

    # log runtime
    run_time = timer() - start
    # calculate loss, mean cross validation score, standard deviation and minimum and maximum cross-validation score
   
    loss = 1 - cv.mean()
    mean_score = cv.mean()
    std_score = cv.std()
    min_score = cv.min()
    max_score = cv.max()

    # export results to csv
    out_file = results_file
    with open(out_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                ITERATION,
                
                loss,
                mean_score,
                std_score,
                min_score,
                max_score,
                run_time,
                space,
            ]
        )

    return loss

In [ ]:
def objective(space):

    global ITERATION
    ITERATION += 1
    start = timer()
    global prediction_all


    # used to store the auc scores for all three parts of MAILOUT TRAIN.
    auc_score = np.empty(3)
    
    #training the XGBClassifier on the combined AZDIAS and CUSTOMER dataset with MAILOUT TRAIN as the validation dataset and early stopping rounds set to 40 
    eval_set = [(validation_data[:, :-1], validation_data[:, -1])]
    xgbcl = xgb.XGBClassifier(colsample_bytree =  space['colsample_bytree'], gamma = space['gamma'], learning_rate = space['learning_rate'], max_depth = space['max_depth'], min_child_weight = space['min_child_weight'], n_estimators = space['n_estimators'], reg_alpha = space['reg_alpha'], scale_pos_weight = space['scale_pos_weight'], subsample = space['subsample'], tree_method = 'gpu_hist', sampling_method = 'gradient_based')
    xgbcl.fit(np.array(train_data.iloc[:,:-1]), np.array(train_data.iloc[:,-1]), early_stopping_rounds=40, eval_metric="auc", eval_set=eval_set)
    
    #trained XGBClassifier model is then used to predict on overall MAILOUT TRAIN data and also the 3 separate parts
    prediction = (xgbcl.predict_proba(validation_data[:,:-1]))[:,1]
    score_a = roc_auc_score(validation_data[:,-1], prediction)
    predictions1 = (xgbcl.predict_proba(mailout_train_sample1[:, :-1]))[:, 1]
    auc_score[0]=roc_auc_score(mailout_train_sample1[:, -1], predictions1)
    predictions2 = (xgbcl.predict_proba(mailout_train_sample2[:, :-1]))[:, 1]
    auc_score[1]=roc_auc_score(mailout_train_sample2[:, -1], predictions2)
    predictions3 = (xgbcl.predict_proba(mailout_train_sample3[:, :-1]))[:, 1]
    auc_score[2]=roc_auc_score(mailout_train_sample3[:, -1], predictions3)
    
     
    #then trained XGBClassifier model is used to predict on MAILOUT TEST and predictions are concatenated to predictions_all_1 variable  
    predictions_xgbcl = (xgbcl.predict_proba(mailout_test))[:, 1]
    predictions_all_1 = pd.concat([predictions_all, predictions_xgbcl], axis=1)
    
   
    # log runtime
    run_time = timer() - start

    # calculate mean loss, mean, min and max auc scores over the 3 parts and overall auc scores on the whole MAILOUT TRAIN dataset

    loss = 1 - auc_score.mean()
    mean_score = auc_score.mean()
    std_score = auc_score.std()
    min_score = auc_score.min()
    max_score = auc_score.max()
    overall_score = score_a

    # export results to csv
    out_file = results_file
    with open(out_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                ITERATION,
                loss,
                overall_score,
                mean_score,
                std_score,
                min_score,
                max_score,
                run_time,
                space
            ]
        )

    return loss